In [67]:
include("piracy.jl")

In [7]:
using SumOfSquares
using ComplexOptInterface
const COI = ComplexOptInterface
using MultivariatePolynomials
const MP = MultivariatePolynomials
function SumOfSquares.vectorization(::Type{COI.HermitianPositiveSemidefiniteConeTriangle})
    return SumOfSquares.HermitianVectorized()
end
function SumOfSquares.matrix_cone(::Type{COI.HermitianPositiveSemidefiniteConeTriangle}, d)
    return COI.HermitianPositiveSemidefiniteConeTriangle(d)
end
function add_constraint(model, p, cone; kws...)
    coefs = MOI.Utilities.vectorize(MP.coefficients(p))
    monos = MP.monomials(p)
    set = JuMP.moi_set(cone, monos; kws...)
    MOI.add_constraint(model, coefs, set)
end
function energy(H, maxdegree, solver;
    cone=NonnegPolyInnerCone{COI.HermitianPositiveSemidefiniteConeTriangle}(),
    kws...
)
    optimizer = MOI.instantiate(solver, with_bridge_type=Float64)
    model = MOI.Bridges.Constraint.SingleBridgeOptimizer{SumOfSquares.Bridges.Constraint.SOSPolynomialBridge{Complex{Float64}}}(
        MOI.Bridges.Constraint.SingleBridgeOptimizer{PolyJuMP.ZeroPolynomialBridge{Complex{Float64}}}(
        MOI.Bridges.Variable.SingleBridgeOptimizer{COI.Bridges.Variable.HermitianToSymmetricPSDBridge{Float64}}(
        MOI.Bridges.Constraint.SingleBridgeOptimizer{COI.Bridges.Constraint.SplitZeroBridge{Float64}}(
        optimizer
    ))))
    γ = MOI.SingleVariable(MOI.add_variable(model))
    cert = SumOfSquares.Certificate.MaxDegree(cone, MonomialBasis, maxdegree)
    cert = SumOfSquares.Certificate.SparseIdeal(MonomialSparsity(1), cert)
    c = add_constraint(model, H - γ, SOSCone(); ideal_certificate=cert, kws...)
    MOI.set(model, MOI.ObjectiveSense(), MOI.MAX_SENSE)
    MOI.set(model, MOI.ObjectiveFunction{MOI.SingleVariable}(), γ)
    MOI.optimize!(model)
    if MOI.get(model, MOI.TerminationStatus()) != MOI.OPTIMAL
        @warn("Termination status: $(MOI.get(model, MOI.TerminationStatus())), $(MOI.get(model, MOI.RawStatusString()))")
    end
    ν = MOI.get(model, SumOfSquares.GramMatrixAttribute(), c)
    MOI.get(model, MOI.ObjectiveValue()), ν
end

energy (generic function with 1 method)

In [2]:
using CondensedMatterSOS
@spin σ[1:2]
function hamiltonian(σ)
    σx, σy, σz = σ
    N = length(σx)
    return -sum(σx[n] * σx[n+1] for n in 1:(N-1)) - σx[N] * σx[1] - sum(σz)
end
H = 1.0 * hamiltonian(σ)

(-2.0 + 0.0im)σˣ₁σˣ₂ + σᶻ₁ + σᶻ₂

In [3]:
function hamiltonian_energy(N, maxdegree, solver; kws...)
    @spin σ[1:N]
    H = 1.0 * hamiltonian(σ)
    energy(H, maxdegree, solver; kws...)
end

hamiltonian_energy (generic function with 1 method)

With hermition matrices

In [8]:
using MosekTools
solver = optimizer_with_attributes(Mosek.Optimizer, MOI.Silent() => true)
bound, ν = hamiltonian_energy(2, 2, solver)
bound

MethodError: MethodError: no method matching *(::MathOptInterface.ScalarAffineFunction{Complex{Float64}}, ::Int64)
Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:529
  *(!Matched::Complex{Bool}, ::Real) at complex.jl:309
  *(!Matched::Missing, ::Number) at missing.jl:115
  ...

In [34]:
ν.Q

7×7 SymMatrix{Complex{Float64}}:
   3.99787e-9+0.0im          …    -1.6652e-9+1.23552e-24im
   4.2734e-21+1.26674e-22im     -1.03389e-20-2.43805e-20im
  2.14353e-22-2.11607e-21im      2.76562e-20+5.27478e-20im
  -1.66501e-9+3.78151e-24im     -4.37705e-23+4.75228e-24im
 -3.96811e-21+6.98069e-23im      1.10735e-20+1.79254e-20im
  2.06645e-22+1.28409e-21im  …   1.46068e-20+8.19064e-21im
   -1.6652e-9-1.23552e-24im       3.39316e-9+0.0im

In [58]:
xx = nothing
SumOfSquares.SparseGramMatrix(x) = (@show typeof(x); global xx; xx = x; error())

In [74]:
xx[2].Q

2×2 SymMatrix{MathOptInterface.ScalarAffineFunction{Complex{Float64}}}:
 x[-5]                    x[-6]+0.0 + 1.0im x[-8]
 x[-6]+0.0 - 1.0im x[-8]  x[-7]

In [60]:
using MosekTools
solver = optimizer_with_attributes(Mosek.Optimizer, MOI.Silent() => true)
bound, ν = hamiltonian_energy(2, 2, solver, sparsity = MonomialSparsity(1))
bound

typeof(x) = Array{GramMatrix{T,MonomialBasis{CondensedMatterSOS.SpinMonomial,Array{CondensedMatterSOS.SpinMonomial,1}},MathOptInterface.ScalarAffineFunction{Complex{Float64}}} where T,1}


ErrorException: 

In [36]:
ν.Q

7×7 SymMatrix{Complex{Float64}}:
   3.99787e-9+0.0im          …    -1.6652e-9+1.23552e-24im
   4.2734e-21+1.26674e-22im     -1.03389e-20-2.43805e-20im
  2.14353e-22-2.11607e-21im      2.76562e-20+5.27478e-20im
  -1.66501e-9+3.78151e-24im     -4.37705e-23+4.75228e-24im
 -3.96811e-21+6.98069e-23im      1.10735e-20+1.79254e-20im
  2.06645e-22+1.28409e-21im  …   1.46068e-20+8.19064e-21im
   -1.6652e-9-1.23552e-24im       3.39316e-9+0.0im

With real symmetric matrices

In [12]:
bound, ν = hamiltonian_energy(2, 2, solver, cone=NonnegPolyInnerCone{MOI.PositiveSemidefiniteConeTriangle}())
bound

-3.9999999997960343

In [17]:
ν.Q

16×16 SymMatrix{Complex{Float64}}:
     0.324992+0.0im   1.66189e-19+0.0im  …     0.0398845+0.0im
  1.66189e-19-0.0im       0.26383+0.0im      6.37605e-17+0.0im
  5.37394e-18-0.0im  -1.64664e-12-0.0im     -4.32608e-17+0.0im
    -0.129003-0.0im  -2.37386e-17-0.0im        -0.129003+0.0im
  5.29416e-19-0.0im     -0.252115-0.0im     -6.06719e-17+0.0im
  2.25684e-18-0.0im  -4.56613e-12-0.0im  …   3.05035e-17+0.0im
    -0.129003-0.0im  -2.34377e-17-0.0im        -0.129003+0.0im
    -0.247885-0.0im   3.02053e-18-0.0im       -0.0101216+0.0im
  2.51721e-12-0.0im    2.7711e-17-0.0im     -1.17356e-11+0.0im
 -2.24666e-18-0.0im     -0.120997-0.0im     -2.75885e-17+0.0im
 -2.51721e-12-0.0im  -2.55555e-17-0.0im  …   1.17356e-11+0.0im
    0.0101216-0.0im   4.84603e-17-0.0im         0.247885+0.0im
 -4.46917e-18-0.0im  -2.23745e-12-0.0im      9.01884e-17+0.0im
 -2.68223e-18-0.0im     0.0927146-0.0im       2.4287e-17+0.0im
  1.12115e-18-0.0im   4.81084e-12-0.0im     -8.46979e-17+0.0im
    0.0398845-0.0im 

In [14]:
bound, ν = hamiltonian_energy(2, 4, solver, cone=NonnegPolyInnerCone{MOI.PositiveSemidefiniteConeTriangle}())
bound

-2.8284271247461685

In [16]:
ν.Q

16×16 SymMatrix{Complex{Float64}}:
     0.324992+0.0im   1.66189e-19+0.0im  …     0.0398845+0.0im
  1.66189e-19-0.0im       0.26383+0.0im      6.37605e-17+0.0im
  5.37394e-18-0.0im  -1.64664e-12-0.0im     -4.32608e-17+0.0im
    -0.129003-0.0im  -2.37386e-17-0.0im        -0.129003+0.0im
  5.29416e-19-0.0im     -0.252115-0.0im     -6.06719e-17+0.0im
  2.25684e-18-0.0im  -4.56613e-12-0.0im  …   3.05035e-17+0.0im
    -0.129003-0.0im  -2.34377e-17-0.0im        -0.129003+0.0im
    -0.247885-0.0im   3.02053e-18-0.0im       -0.0101216+0.0im
  2.51721e-12-0.0im    2.7711e-17-0.0im     -1.17356e-11+0.0im
 -2.24666e-18-0.0im     -0.120997-0.0im     -2.75885e-17+0.0im
 -2.51721e-12-0.0im  -2.55555e-17-0.0im  …   1.17356e-11+0.0im
    0.0101216-0.0im   4.84603e-17-0.0im         0.247885+0.0im
 -4.46917e-18-0.0im  -2.23745e-12-0.0im      9.01884e-17+0.0im
 -2.68223e-18-0.0im     0.0927146-0.0im       2.4287e-17+0.0im
  1.12115e-18-0.0im   4.81084e-12-0.0im     -8.46979e-17+0.0im
    0.0398845-0.0im 

# Draft

The rest of this notebooks are drafts that should be deleted

In [18]:
H

(-2.0 + 0.0im)σˣ₁σˣ₂ + -σᶻ₁ + -σᶻ₂

In [46]:
monomials(σ[1], 1:3)

15-element Array{CondensedMatterSOS.SpinMonomial,1}:
 σˣ₁
 σʸ₁
 σᶻ₁
 σˣ₂
 σʸ₂
 σᶻ₂
 σˣ₁σˣ₂
 σˣ₁σʸ₂
 σˣ₁σᶻ₂
 σʸ₁σˣ₂
 σʸ₁σʸ₂
 σʸ₁σᶻ₂
 σᶻ₁σˣ₂
 σᶻ₁σʸ₂
 σᶻ₁σᶻ₂

In [19]:
import MultivariatePolynomials
const MP = MultivariatePolynomials
v = [1, σx[1], σx[2], σy[1], σy[2], σz[1], σz[2]]

7-element Array{CondensedMatterSOS.SpinTerm{Int64},1}:
 1
 σˣ₁
 σˣ₂
 σʸ₁
 σʸ₂
 σᶻ₁
 σᶻ₂

In [29]:
cone = SOSCone()
certificate = SumOfSquares.Certificate.MaxDegree(cone, MonomialBasis, 2)
Certificate.sparsity(H, MonomialSparsity(0), certificate)

5-element Array{MonomialBasis{CondensedMatterSOS.SpinMonomial,Array{CondensedMatterSOS.SpinMonomial,1}},1}:
 MonomialBasis{CondensedMatterSOS.SpinMonomial,Array{CondensedMatterSOS.SpinMonomial,1}}(CondensedMatterSOS.SpinMonomial[σᶻ₂, 1])
 MonomialBasis{CondensedMatterSOS.SpinMonomial,Array{CondensedMatterSOS.SpinMonomial,1}}(CondensedMatterSOS.SpinMonomial[σᶻ₁, 1])
 MonomialBasis{CondensedMatterSOS.SpinMonomial,Array{CondensedMatterSOS.SpinMonomial,1}}(CondensedMatterSOS.SpinMonomial[σˣ₁, σˣ₂])
 MonomialBasis{CondensedMatterSOS.SpinMonomial,Array{CondensedMatterSOS.SpinMonomial,1}}(CondensedMatterSOS.SpinMonomial[σʸ₂])
 MonomialBasis{CondensedMatterSOS.SpinMonomial,Array{CondensedMatterSOS.SpinMonomial,1}}(CondensedMatterSOS.SpinMonomial[σʸ₁])

min_σ H(σ) = -2√2

max γ s.t. H >= γ -> H - γ = v' Q v

In [5]:
function _mat(x::Vector, n)
    Q = Matrix{eltype(x)}(undef, n, n)
    k = 0
    for j in 1:n
        for i in 1:j
            k += 1
            Q[i, j] = Q[j, i] = x[k]
        end
    end
    return Q
end
import MathOptInterface
const MOI = MathOptInterface
function _mapcoef(op, f::MOI.ScalarAffineFunction)
    MOI.ScalarAffineFunction([
        MOI.ScalarAffineTerm(op(t.coefficient), t.variable_index) for t in f.terms
    ], op(f.constant))
end

_mapcoef (generic function with 1 method)

# Real

In [ ]:
function operate!(::typeof(*), ::Type{T}, α::T,
                  f::MOI.ScalarAffineFunction) where T
    map_terms!(term -> operate_term(*, term, α), f)
    rmul!(f.constants, α)
    return f
end
function operate(::typeof(*), ::Type{T}, α::T, f::TypedLike{T}) where T
    return operate!(*, T, copy(f), α)
end

In [45]:
using SumOfSquares
Base.promote_rule(::Type{T}, ::Type{MOI.SingleVariable}) where {T<:Number} = MOI.ScalarAffineFunction{T}
PolyJuMP.non_constant(a::Vector{<:MOI.AbstractFunction}) = a
function MultivariatePolynomials.variables(monos::Vector{CondensedMatterSOS.SpinMonomial})
    vars = Set{CondensedMatterSOS.SpinVariable}()
    for mono in monos
        union!(vars, variables(mono))
    end
    return sort(collect(vars), rev=true)
end
function Base.convert(::Type{MOI.ScalarAffineFunction{T}}, f::MOI.ScalarAffineFunction) where T
    return MOI.ScalarAffineFunction(
        [MOI.ScalarAffineTerm(convert(T, t.coefficient), t.variable_index) for t in f.terms],
        convert(T, f.constant)
    )
end
function MOI.Utilities.operate(::typeof(*), ::Type{Complex{Float64}}, a::Complex{Float64}, f::MOI.ScalarAffineFunction{Float64})
    MOI.Utilities.operate(*, Complex{Float64}, a, convert(MOI.ScalarAffineFunction{Complex{Float64}}, f))
end
MultivariatePolynomials.variables(p::CondensedMatterSOS.SpinPolynomial) = variables(monomials(p))
Base.copy(t::CondensedMatterSOS.SpinTerm) = CondensedMatterSOS.SpinTerm(t.coefficient, t.monomial)
Base.:*(f::MOI.ScalarAffineFunction{Complex{Float64}}, a::Number) = f * convert(Complex{Float64}, a)
Base.:*(f::MOI.ScalarAffineFunction{Complex{Float64}}, a::Complex{Float64}) = MOI.Utilities.operate(*, Complex{Float64}, a, f)
Base.:+(a::Int, f::MOI.ScalarAffineFunction{Complex{Float64}}) = convert(Complex{Float64}, a) + f
Base.:+(a::Complex{Float64}, f::MOI.ScalarAffineFunction{Complex{Float64}}) = MOI.Utilities.operate(+, Complex{Float64}, a, f)

In [6]:
function add_constraint(model, p, cone; kws...)
    coefs = MOI.Utilities.vectorize(PolyJuMP.non_constant_coefficients(p))
    monos = MP.monomials(p)
    set = JuMP.moi_set(cone, monos; kws...)
    MOI.add_constraint(model, coefs, set)
end

add_constraint (generic function with 1 method)

In [12]:
using SumOfSquares
using ComplexOptInterface
const COI = ComplexOptInterface
using MosekTools
solver = optimizer_with_attributes(Mosek.Optimizer, MOI.Silent() => true)
optimizer = MOI.instantiate(solver, with_bridge_type=Float64)
model = MOI.Bridges.Constraint.SingleBridgeOptimizer{SumOfSquares.Bridges.Constraint.SOSPolynomialBridge{Complex{Float64}}}(
    MOI.Bridges.Constraint.SingleBridgeOptimizer{PolyJuMP.ZeroPolynomialBridge{Complex{Float64}}}(
    MOI.Bridges.Constraint.SingleBridgeOptimizer{COI.Bridges.Constraint.SplitZeroBridge{Float64}}(
    optimizer
)))
γ = MOI.SingleVariable(MOI.add_variable(model))
add_constraint(model, H - γ, SOSCone(), ideal_certificate=SumOfSquares.Certificate.MaxDegree(SOSCone(), MonomialBasis, 2))
MOI.set(model, MOI.ObjectiveSense(), MOI.MAX_SENSE)
MOI.set(model, MOI.ObjectiveFunction{MOI.SingleVariable}(), γ)
MOI.optimize!(model)
MOI.get(model, MOI.ObjectiveValue())

-3.9999999997960343

In [38]:
function energy(H, maxdegree, cone)
    solver = optimizer_with_attributes(Mosek.Optimizer, MOI.Silent() => true)
    optimizer = MOI.instantiate(solver, with_bridge_type=Float64)
    model = MOI.Bridges.Constraint.SingleBridgeOptimizer{SumOfSquares.Bridges.Constraint.SOSPolynomialBridge{Complex{Float64}}}(
        MOI.Bridges.Constraint.SingleBridgeOptimizer{PolyJuMP.ZeroPolynomialBridge{Complex{Float64}}}(
        MOI.Bridges.Variable.SingleBridgeOptimizer{COI.Bridges.Variable.HermitianToSymmetricPSDBridge{Float64}}(
        MOI.Bridges.Constraint.SingleBridgeOptimizer{COI.Bridges.Constraint.SplitZeroBridge{Float64}}(
        optimizer
    ))))
    γ = MOI.SingleVariable(MOI.add_variable(model))
    cert = SumOfSquares.Certificate.MaxDegree(cone, MonomialBasis, maxdegree)
    add_constraint(model, H - γ, SOSCone(), ideal_certificate=cert)
    MOI.set(model, MOI.ObjectiveSense(), MOI.MAX_SENSE)
    MOI.set(model, MOI.ObjectiveFunction{MOI.SingleVariable}(), γ)
    MOI.optimize!(model)
    if MOI.get(model, MOI.TerminationStatus()) != MOI.OPTIMAL
        @warn("Termination status: $(MOI.get(model, MOI.TerminationStatus())), $(MOI.get(model, MOI.RawStatusString()))")
    end
    MOI.get(model, MOI.ObjectiveValue())
end

energy (generic function with 3 methods)

In [33]:
energy(H, 2, NonnegPolyInnerCone{MOI.PositiveSemidefiniteConeTriangle}())

-3.9999999997960343

In [34]:
energy(H, 4, NonnegPolyInnerCone{MOI.PositiveSemidefiniteConeTriangle}())

-2.8284271247461685

In [114]:
function SumOfSquares.matrix_cone(::Type{COI.HermitianPositiveSemidefiniteConeTriangle}, d)
    return COI.HermitianPositiveSemidefiniteConeTriangle(d)
end
using MutableArithmetics
const MA = MutableArithmetics
function MOI.Utilities._add_sub_affine_terms(
    op::Union{typeof(+), typeof(-)}, terms::Vector{MOI.ScalarAffineTerm{T}},
    α::T, f::MOI.SingleVariable) where T
    push!(terms, MOI.ScalarAffineTerm(op(α), f.variable))
end
MOI.Utilities._constant(::Type{T}, ::MOI.SingleVariable) where T = zero(T)
function SumOfSquares.add_gram_matrix(
        model::MOI.ModelLike,
        matrix_cone_type::Type{COI.HermitianPositiveSemidefiniteConeTriangle},
        basis::AbstractPolynomialBasis, T::Type{<:Complex})
    n = length(basis)
    Q, cQ = MOI.add_constrained_variables(model, SumOfSquares.matrix_cone(matrix_cone_type, n))
    N = MOI.dimension(MOI.PositiveSemidefiniteConeTriangle(n))
    C = [zero(MOI.ScalarAffineFunction{T}) for i in 1:N]
    k_real = 0
    k_imag = 0
    for j in 1:n
        for i in 1:j
            k_real += 1
            MA.mutable_operate!(MA.add_mul, C[k_real], one(T), MOI.SingleVariable(Q[k_real]))
            if i != j
                k_imag += 1
                MA.mutable_operate!(MA.add_mul, C[k_real], one(T) * im, MOI.SingleVariable(Q[N + k_imag]))
            end
        end
    end
    q = SumOfSquares.build_gram_matrix(C, basis, T)
    return q, Q, cQ
end

In [108]:
q = SumOfSquares.SymMatrix([1.0, 1.0 + im, 2.0], 2)
function Base.getindex(Q::SymMatrix, i::Integer, j::Integer)
    if i <= j
        return Q.Q[MultivariateMoments.trimap(j, i)]
    else
        return conj(Q[j, i])
    end
end
function COI.Bridges.Constraint.operate_coefficients(f, T::Type, func::MOI.ScalarAffineFunction)
    return MOI.ScalarAffineFunction(
        [COI.Bridges.Constraint.operate_coefficient(f, T, term) for term in func.terms],
        f(func.constant)
    )
end
# We assume MOI variables are real
Base.conj(func::MOI.Utilities.TypedLike{T}) where T = COI.Bridges.Constraint.operate_coefficients(conj, T, func)
q

2×2 SymMatrix{Complex{Float64}}:
 1.0+0.0im  1.0+1.0im
 1.0-1.0im  2.0+0.0im

In [92]:
default_name(vi) = string("x[", vi.value, "]")
function function_string(::Type{JuMP.REPLMode}, v::MOI.VariableIndex, name = default_name)
    var_name = name(v)
    if !isempty(var_name)
        return var_name
    else
        return "noname"
    end
end
function function_string(::Type{JuMP.IJuliaMode}, v::MOI.VariableIndex, name = default_name)
    var_name = name(v)
    if !isempty(var_name)
        # TODO: This is wrong if variable name constains extra "]"
        return replace(replace(var_name, "[" => "_{", count = 1), "]" => "}")
    else
        return "noname"
    end
end
# Whether something is zero or not for the purposes of printing it
# oneunit is useful e.g. if coef is a Unitful quantity. The second `abs` is import if it is complex.
_is_zero_for_printing(coef) = abs(coef) < 1e-10 * abs(oneunit(coef))
# Whether something is one or not for the purposes of printing it.
_is_one_for_printing(coef) = _is_zero_for_printing(abs(coef) - oneunit(coef))
_is_one_for_printing(coef::Complex) = _is_one_for_printing(real(coef)) && _is_zero_for_printing(imag(coef))
_sign_string(coef) = coef < zero(coef) ? " - " : " + "
_sign_string(coef) = coef < zero(coef) ? " - " : " + "
function function_string(mode, func::MOI.ScalarAffineFunction, show_constant=true)
    # If the expression is empty, return the constant (or 0)
    if isempty(func.terms)
        return show_constant ? JuMP._string_round(MOI.constant(func)) : "0"
    end

    term_str = Vector{String}(undef, 2length(func.terms))
    elm = 1

    for term in func.terms
        pre = _is_one_for_printing(term.coefficient) ? "" : JuMP._string_round(term.coefficient) * " "

        term_str[2 * elm - 1] = "+"
        term_str[2 * elm] = string(pre, function_string(mode, term.variable_index))
        elm += 1
    end

    if elm == 1
        # Will happen with cancellation of all terms
        # We should just return the constant, if its desired
        return show_constant ? JuMP._string_round(a.constant) : "0"
    else
        # Correction for very first term - don't want a " + "/" - "
        term_str[1] = (term_str[1] == " - ") ? "-" : ""
        ret = join(term_str[1 : 2 * (elm - 1)])
        if !_is_zero_for_printing(MOI.constant(func)) && show_constant
            ret = string(ret, "+",
                         JuMP._string_round(MOI.constant(constant)))
        end
        return ret
    end
end
function Base.show(io::IO, f::MOI.AbstractScalarFunction)
    print(io, function_string(JuMP.REPLMode, f))
end
function Base.show(io::IO, ::MIME"text/latex", f::MOI.AbstractScalarFunction)
    print(io, JuMP._wrap_in_math_mode(function_string(JuMP.IJuliaMode, f)))
end

In [109]:
x = MOI.SingleVariable(MOI.VariableIndex(1))
conj((im) * x)

0 - 1im x[1]

In [115]:
energy(H, 2, NonnegPolyInnerCone{COI.HermitianPositiveSemidefiniteConeTriangle}())

-2.828427118189658

In [11]:
using MosekTools
optimizer = Mosek.Optimizer()
MOI.set(optimizer, MOI.Silent(), true)
n = length(v)
x, cx = MOI.add_constrained_variables(optimizer, MOI.PositiveSemidefiniteConeTriangle(n))
γ = MOI.SingleVariable(MOI.add_variable(optimizer))
F = MOI.ScalarAffineFunction{Complex{Float64}}
Q = _mat(F.(MOI.SingleVariable.(x)), n)
gram = sum(Q[i, j] * (1.0 * monos[i] * monos[j]) for i in 1:n for j in 1:n)
z = H - convert(F, γ) - gram
for c in MP.coefficients(z)
    for a in [_mapcoef(real, c), _mapcoef(imag, c)]
        MOI.Utilities.normalize_and_add_constraint(optimizer, a, MOI.EqualTo(0.0))
    end
end
MOI.set(optimizer, MOI.ObjectiveSense(), MOI.MAX_SENSE)
MOI.set(optimizer, MOI.ObjectiveFunction{MOI.SingleVariable}(), γ)
MOI.optimize!(optimizer)
MOI.get(optimizer, MOI.ObjectiveValue())

-3.999999999796037

# Complex

In [13]:
using CSDP
optimizer = MOI.instantiate(CSDP.Optimizer, with_bridge_type=Float64)
n = length(v)
x, cx = MOI.add_constrained_variables(optimizer, MOI.PositiveSemidefiniteConeTriangle(2n))
X = _mat(MOI.SingleVariable.(x), 2n)
X11 = X[1:n, 1:n]
X22 = X[n .+ (1:n), n .+ (1:n)]
X12 = X[1:n, n .+ (1:n)]
for j in 1:n
    for i in 1:j
        MOI.add_constraint(optimizer, MOI.Utilities.operate(-, Float64, X11[i, j], X22[i, j]), MOI.EqualTo(0.0))
        if i == j
            MOI.add_constraint(optimizer, convert(MOI.ScalarAffineFunction{Float64}, X12[i, j]), MOI.EqualTo(0.0))
        else
            MOI.add_constraint(optimizer, MOI.Utilities.operate(+, Float64, X12[i, j], X12[j, i]), MOI.EqualTo(0.0))
        end
    end
end
γ = MOI.SingleVariable(MOI.add_variable(optimizer))
F = MOI.ScalarAffineFunction{Complex{Float64}}
Q_real = F.(X11)
Q_comp = F.(X12)
Q = Q_real + (-1.0*im) * Q_comp
gram = sum(Q[i, j] * (1.0 * monos[i] * monos[j]) for i in 1:n for j in 1:n)
z = H - convert(F, γ) - gram
duals = Dict()
for term in MP.terms(z)
    duals[MP.monomial(term)] = map([real, imag]) do f
        a = _mapcoef(f, MP.coefficient(term))
        MOI.Utilities.normalize_and_add_constraint(optimizer, a, MOI.EqualTo(0.0))
    end
end
MOI.set(optimizer, MOI.ObjectiveSense(), MOI.MAX_SENSE)
MOI.set(optimizer, MOI.ObjectiveFunction{MOI.SingleVariable}(), γ)
MOI.optimize!(optimizer)
MOI.get(optimizer, MOI.ObjectiveValue())

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.00e+00 Pobj: -5.6507691e+01 Ad: 4.01e-01 Dobj: -1.7634637e+00 
Iter:  2 Ap: 1.00e+00 Pobj: -5.5377524e+01 Ad: 9.54e-01 Dobj: -3.1515529e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -2.4963595e+01 Ad: 8.83e-01 Dobj: -5.2930472e-01 
Iter:  4 Ap: 1.00e+00 Pobj: -6.2669103e+00 Ad: 7.62e-01 Dobj: -1.2742768e+00 
Iter:  5 Ap: 1.00e+00 Pobj: -3.5102614e+00 Ad: 8.76e-01 Dobj: -2.3614414e+00 
Iter:  6 Ap: 1.00e+00 Pobj: -3.0661231e+00 Ad: 8.87e-01 Dobj: -2.7653758e+00 
Iter:  7 Ap: 1.00e+00 Pobj: -2.8484566e+00 Ad: 8.92e-01 Dobj: -2.8141101e+00 
Iter:  8 Ap: 1.00e+00 Pobj: -2.8301498e+00 Ad: 9.38e-01 Dobj: -2.8268972e+00 
Iter:  9 Ap: 1.00e+00 Pobj: -2.8285576e+00 Ad: 9.86e-01 Dobj: -2.8283564e+00 
Iter: 10 Ap: 9.75e-01 Pobj: -2.8284373e+00 Ad: 9.98e-01 Dobj: -2.8284264e+00 
Iter: 11 Ap: 1.00e+00 Pobj: -2.8284276e+00 Ad: 1.00e+00 Dobj: -2.8284271e+00 
Iter: 12 Ap: 9.58e-01 Pobj: -2.8284271e+00 Ad: 9.60e-

-2.828427145315304

In [36]:
using CSDP
optimizer = MOI.instantiate(CSDP.Optimizer, with_bridge_type=Float64)
n = length(v)
x, cx = MOI.add_constrained_variables(optimizer, MOI.PositiveSemidefiniteConeTriangle(2n))
X = _mat(MOI.SingleVariable.(x), 2n)
X11 = X[1:n, 1:n]
X22 = X[n .+ (1:n), n .+ (1:n)]
X12 = X[1:n, n .+ (1:n)]
for j in 1:n
    for i in 1:j
        MOI.add_constraint(optimizer, MOI.Utilities.operate(-, Float64, X11[i, j], X22[i, j]), MOI.EqualTo(0.0))
        if i == j
            MOI.add_constraint(optimizer, convert(MOI.ScalarAffineFunction{Float64}, X12[i, j]), MOI.EqualTo(0.0))
        else
            MOI.add_constraint(optimizer, MOI.Utilities.operate(+, Float64, X12[i, j], X12[j, i]), MOI.EqualTo(0.0))
        end
    end
end
γ = MOI.SingleVariable(MOI.add_variable(optimizer))
F = MOI.ScalarAffineFunction{Complex{Float64}}
Q_real = F.(X11)
Q_comp = F.(X12)
Q = Q_real + (-1.0*im) * Q_comp
gram = sum(Q[i, j] * (1.0 * monos[i] * monos[j]) for i in 1:n for j in 1:n)
z = H - convert(F, γ) - gram
duals = Dict()
for term in MP.terms(z)
    duals[MP.monomial(term)] = map([real, imag]) do f
        a = _mapcoef(f, MP.coefficient(term))
        MOI.Utilities.normalize_and_add_constraint(optimizer, a, MOI.EqualTo(0.0))
    end
end
MOI.set(optimizer, MOI.ObjectiveSense(), MOI.MAX_SENSE)
MOI.set(optimizer, MOI.ObjectiveFunction{MOI.SingleVariable}(), γ)
MOI.optimize!(optimizer)

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.00e+00 Pobj: -5.6507691e+01 Ad: 4.01e-01 Dobj: -1.7634637e+00 
Iter:  2 Ap: 1.00e+00 Pobj: -5.5377524e+01 Ad: 9.54e-01 Dobj: -3.1515529e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -2.4963595e+01 Ad: 8.83e-01 Dobj: -5.2930472e-01 
Iter:  4 Ap: 1.00e+00 Pobj: -6.2669103e+00 Ad: 7.62e-01 Dobj: -1.2742768e+00 
Iter:  5 Ap: 1.00e+00 Pobj: -3.5102614e+00 Ad: 8.76e-01 Dobj: -2.3614414e+00 
Iter:  6 Ap: 1.00e+00 Pobj: -3.0661231e+00 Ad: 8.87e-01 Dobj: -2.7653758e+00 
Iter:  7 Ap: 1.00e+00 Pobj: -2.8484566e+00 Ad: 8.92e-01 Dobj: -2.8141101e+00 
Iter:  8 Ap: 1.00e+00 Pobj: -2.8301498e+00 Ad: 9.38e-01 Dobj: -2.8268972e+00 
Iter:  9 Ap: 1.00e+00 Pobj: -2.8285576e+00 Ad: 9.86e-01 Dobj: -2.8283564e+00 
Iter: 10 Ap: 9.75e-01 Pobj: -2.8284373e+00 Ad: 9.98e-01 Dobj: -2.8284264e+00 
Iter: 11 Ap: 1.00e+00 Pobj: -2.8284276e+00 Ad: 1.00e+00 Dobj: -2.8284271e+00 
Iter: 12 Ap: 9.58e-01 Pobj: -2.8284271e+00 Ad: 9.60e-

15.998771905899048

In [30]:
@show MOI.get(optimizer, MOI.TerminationStatus())

MOI.get(optimizer, MOI.TerminationStatus()) = MathOptInterface.OPTIMAL


OPTIMAL::TerminationStatusCode = 1

In [31]:
-2 * √2

-2.8284271247461903

In [32]:
MOI.get(optimizer, MOI.ObjectiveValue())

-2.828427145315304

In [14]:
MOI.get(optimizer, MOI.SolveTime())

4.379154205322266

In [15]:
Mfull = round.(_mat(MOI.get(optimizer, MOI.ConstraintDual(), cx), 2n), digits=6)
M11 = Mfull[1:n, 1:n]
M12 = Mfull[1:n, n .+ (1:n)]
M12 + im * M12

7×7 Array{Complex{Float64},2}:
       0.0+0.0im       0.353553+0.353553im  …  0.0+0.0im  0.0+0.0im
 -0.353553-0.353553im       0.0+0.0im          0.0+0.0im  0.0+0.0im
       0.0+0.0im            0.0+0.0im          0.0+0.0im  0.0+0.0im
       0.0+0.0im           -0.0-0.0im          0.0+0.0im  0.0+0.0im
      -0.0-0.0im            0.0+0.0im          0.0+0.0im  0.0+0.0im
       0.0+0.0im            0.0+0.0im       …  0.0+0.0im  0.0+0.0im
       0.0+0.0im            0.0+0.0im          0.0+0.0im  0.0+0.0im

In [34]:
Mfull = round.(_mat(MOI.get(optimizer, MOI.ConstraintDual(), cx), 2n), digits=6)
M11 = Mfull[1:n, 1:n]
M12 = Mfull[1:n, n .+ (1:n)]
M12 + im * M12

7×7 Array{Complex{Float64},2}:
       0.0+0.0im       0.353553+0.353553im  …  0.0+0.0im  0.0+0.0im
 -0.353553-0.353553im       0.0+0.0im          0.0+0.0im  0.0+0.0im
       0.0+0.0im            0.0+0.0im          0.0+0.0im  0.0+0.0im
       0.0+0.0im           -0.0-0.0im          0.0+0.0im  0.0+0.0im
      -0.0-0.0im            0.0+0.0im          0.0+0.0im  0.0+0.0im
       0.0+0.0im            0.0+0.0im       …  0.0+0.0im  0.0+0.0im
       0.0+0.0im            0.0+0.0im          0.0+0.0im  0.0+0.0im

In [17]:
using MultivariateMoments
function _moment_matrix(μ::MultivariateMoments.Measure{T}, X) where T
    function getmom(i, j)
        x = X[i] * X[j]
        for m in moments(μ)
            if monomial(m) == monomial(x)
                return MP.coefficient(x) * moment_value(m)
            end
        end
        throw(ArgumentError("μ does not have the moment $(x)"))
    end
    return MultivariateMoments.MomentMatrix{T}(getmom, X)
end

┌ Info: Precompiling MultivariateMoments [f4abf1af-0426-5881-a0da-e2f168889b5e]
└ @ Base loading.jl:1260


_moment_matrix (generic function with 1 method)

In [18]:
a = Complex{Float64}[]
m = MP.monomialtype(H)[]
for (key, value) in duals
    push!(m, key)
    real_dual = MOI.get(optimizer, MOI.ConstraintDual(), value[1])
    comp_dual = MOI.get(optimizer, MOI.ConstraintDual(), value[2])
    push!(a, real_dual + im * comp_dual)
end
μ = measure(a, m)
ν = _moment_matrix(μ, v)

MomentMatrix{Complex{Float64},MultivariateBases.MonomialBasis{CondensedMatterSOS.SpinMonomial,Array{CondensedMatterSOS.SpinMonomial,1}}}(Complex{Float64}[1.0000000007926881 + 0.0im 0.0 - 0.7071061572962712im … 0.0 - 0.0im 0.0 - 0.0im; 0.0 - 0.7071061572962712im 1.0000000007926881 + 0.0im … 0.0 - 0.0im 0.0 - 0.0im; … ; 0.0 - 0.0im 0.0 - 0.0im … 1.0000000007926881 + 0.0im 0.7071061572962712 + 0.0im; 0.0 - 0.0im 0.0 - 0.0im … 0.7071061572962712 + 0.0im 1.0000000007926881 + 0.0im], MultivariateBases.MonomialBasis{CondensedMatterSOS.SpinMonomial,Array{CondensedMatterSOS.SpinMonomial,1}}(CondensedMatterSOS.SpinMonomial[σˣ₁, σʸ₁, σᶻ₁, σˣ₂, σʸ₂, σᶻ₂, 1]), nothing)

In [19]:
round.(ν.Q, digits = 1)

7×7 Array{Complex{Float64},2}:
 1.0+0.0im   0.0-0.7im   0.0-0.0im  …   0.0-0.0im   0.0-0.0im  0.0-0.0im
 0.0-0.7im   1.0+0.0im  -0.0+0.0im     -0.7-0.0im   0.0-0.0im  0.0-0.0im
 0.0-0.0im  -0.0+0.0im   1.0+0.0im      0.0-0.0im   0.5+0.0im  0.7+0.0im
 0.7+0.0im   0.0+0.0im   0.0-0.0im      0.0-0.7im   0.0-0.0im  0.0-0.0im
 0.0-0.0im  -0.7-0.0im   0.0-0.0im      1.0+0.0im  -0.0+0.0im  0.0-0.0im
 0.0-0.0im   0.0-0.0im   0.5+0.0im  …  -0.0+0.0im   1.0+0.0im  0.7+0.0im
 0.0-0.0im   0.0-0.0im   0.7+0.0im      0.0-0.0im   0.7+0.0im  1.0+0.0im

In [20]:
using LinearAlgebra
Γ = Hermitian([
    1 0 0   0     0     1/2 1/2
    0 1 1/2 1/2im 0     0   0
    0 0 1   0     1/2im 0   0
    0 0 0   1    -1/2   0   0
    0 0 0   0     1     0   0
    0 0 0   0     0     1   1
    0 0 0   0     0     0   1
])

7×7 Hermitian{Complex{Float64},Array{Complex{Float64},2}}:
 1.0+0.0im  0.0+0.0im  0.0+0.0im  …   0.0+0.0im  0.5+0.0im  0.5+0.0im
 0.0-0.0im  1.0+0.0im  0.5+0.0im      0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0-0.0im  0.5-0.0im  1.0+0.0im      0.0-0.5im  0.0+0.0im  0.0+0.0im
 0.0-0.0im  0.0+0.5im  0.0-0.0im     -0.5+0.0im  0.0+0.0im  0.0+0.0im
 0.0-0.0im  0.0-0.0im  0.0+0.5im      1.0+0.0im  0.0+0.0im  0.0+0.0im
 0.5-0.0im  0.0-0.0im  0.0-0.0im  …   0.0-0.0im  1.0+0.0im  1.0+0.0im
 0.5-0.0im  0.0-0.0im  0.0-0.0im      0.0-0.0im  1.0-0.0im  1.0+0.0im